In [ ]:
import pandas as pd
import os

pasta = r"G:\Meu Drive\Estudos\Mestrado\Github\masters\bases_originais\Clima"

dfs = []

for arquivo in os.listdir(pasta):
    if arquivo.endswith('.CSV') and 'JOAO PESSOA' in arquivo.upper():
        caminho_completo = os.path.join(pasta, arquivo)

        df = pd.read_csv(
            caminho_completo,
            encoding='latin-1',
            skiprows=8,
            sep=';'
        )

        # garante que existem pelo menos 8 colunas
        if df.shape[1] < 8:
            continue

        # mantém 1ª, 2ª, 3ª e 8ª colunas
        df = df.iloc[:, [0, 1, 2, 7]]

        # renomeia as duas primeiras
        df.columns = ['DATA', 'HORA', df.columns[2], df.columns[3]]

        dfs.append(df)

df_final = pd.concat(dfs, ignore_index=True)


In [ ]:
import numpy as np

df_final['TEMPERATURA'] = (
    df_final['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)']
    .replace(['-9999', -999, 9999, 999.9], np.nan)
)

df_final['PRECIPITAÇÃO'] = (
    df_final['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']
    .replace(['-9999', -999, 9999, 999.9], np.nan)
)

In [ ]:
df_final = df_final[['DATA',	'HORA', 'TEMPERATURA', 'PRECIPITAÇÃO']]
df_final

In [ ]:
import pandas as pd
import numpy as np

df = df_final.copy()
df = df[~df['DATA'].astype(str).str.contains('2018|2019|2020|2021', na=False)]

# DATA como date
df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce').dt.date

# numéricas
colunas_numericas = ['TEMPERATURA', 'PRECIPITAÇÃO']
df[colunas_numericas] = df[colunas_numericas].apply(
    pd.to_numeric, errors='coerce'
)

df_diario = (
    df
    .groupby('DATA', as_index=False)
    .agg(
        temp_max=('TEMPERATURA', 'max'),
        temp_min=('TEMPERATURA', 'min'),
        temp_media=('TEMPERATURA', 'mean'),
        precipitacao_total=('PRECIPITAÇÃO', 'sum')
    )
)
df_diario = df_diario.sort_values('DATA')

In [ ]:
df_diario.info()

In [ ]:
df_diario.to_csv(r"G:\Meu Drive\Estudos\Mestrado\Github\masters\bases_tratadas\weather_dataset_2022-2024.csv", index=False, sep=';')
